In [ ]:
"""
This script can be used to predict the pixel-wise density of
the PS points for a given region. It uses a trained model and
the input data consists of long-term coherence that can be downloaded
by the script and infrastructure map thats generated by processing
an OSM file that has to be already downloaded.
The script generates prediction chips and saves them
as a geotiff file.
"""

In [ ]:
# check Python version (3.9 needed)
# !python --version # jupyter
# import platform

In [ ]:
# Import modules
import os
import sys
import warnings
import time
import math

In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
from torchgeo.datasets import (
    stack_samples,
)
from torchgeo.samplers import GridGeoSampler

In [ ]:
from lightning.pytorch import seed_everything

In [ ]:
from ps_predictions.utils.torchgeo_classes_def import PixelwiseRegressionTask

In [ ]:
from ps_predictions.utils.torchgeo_fun_to_read_input import (
    scale_coh,
    RasterDataset_imgs,
)
from ps_predictions.utils.prediction_plots import (
    georreferenced_chip_generator,
    merge_georeferenced_chips,
)

In [ ]:
from ps_predictions.infrastructure_map_generation import generate_infrastructure_map

print(f"Python version {platform.python_version()}")

In [ ]:
# Check if torch with cuda enabled and working
# torch.zeros(1).cuda()
torch.cuda.is_available()

In [ ]:
# Ignore warning about number of warnings
# See https://github.com/Lightning-AI/lightning/issues/10182
warnings.filterwarnings("ignore", ".*does not have many workers.*")

In [ ]:
# Define functions #
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    This function calculates the haversine distance between two points on the Earth's surface.

    Arguments:
        lat1 (float): Latitude of the first point in degrees
        lon1 (float): Longitude of the first point in degrees
        lat2 (float): Latitude of the second point in degrees
        lon2 (float): Longitude of the second point in degrees

    Returns:
        float: The haversine distance between the two points in kilometers
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

Set-up region to be analysed #

In [ ]:
# !!! FOR NOW IT ONLY WORKS IF THE REGION IS WITHIN ONE BY ONE DEGREE TILE (TO BE UPDATED) !!!
x = [18.53, 18.9]
y = [50.05, 50.5]

In [ ]:
# Calculate distances
lat_distance = haversine_distance(y[0], x[0], y[1], x[0])
lon_distance = haversine_distance(y[0], x[0], y[0], x[1])

In [ ]:
# Calculate area in square kilometers
area_km2 = lat_distance * lon_distance
print(f"Area: {area_km2:.2f} km²")

Set-up variables #

In [ ]:
root = "/mnt/d/ML_model_data_paper/"

In [ ]:
# Path for saving results (such as model checkpoints)
path_prefix = os.path.join(root + "testing_new_strucutre")

In [ ]:
experiment_name = "testing_new_strucutre"
experiment_dir = os.path.join(path_prefix, experiment_name)
# print(experiment_dir)

In [ ]:
os.makedirs(experiment_dir, exist_ok=True)

In [ ]:
# Define the path for the clipped O5M file
clipped_osm_pbf = os.path.join(experiment_dir, "slaskie-latest.osm.pbf")

Set-up model parameters #

# Constant parameters
in_channels = 2  # Number of channels in input image
ignore_index = None  # Optional integer class index to ignore in the loss and metrics
lr_patience = 10  # Patience for learning rate scheduler
min_delta_lr = 0.01
patience = 15  # Patience for early stopping
min_delta = (
    0.01  # when change in val_loss less than that for appropriate number of epochs
)
# early stopping will be triggered
max_epochs = 300  # maximum number of epochs

gpu_id = 0
device = torch.device(f"cuda:{gpu_id}")
num_dataloader_workers = 0

param_const = [
    in_channels,
    ignore_index,
    lr_patience,
    min_delta_lr,
    patience,
    min_delta,
    max_epochs,
    gpu_id,
    device,
    num_dataloader_workers,
]

In [ ]:
# Modifiable parameters
param_opts = [
    [16, 64, None, "resnet34", None, "mae", 0.00776601, 0.9, 0.99, 0.01],
]
patch_size = param_opts[0][1]
batch_size = param_opts[0][0]

In [ ]:
# Define the patch size and batch size for the model
patch_size = 64
batch_size = 16

In [ ]:
# Select overlap parameter for merging patches in prediction
overlap = 8

Read trained model #

In [ ]:
# Read trained model
model = PixelwiseRegressionTask.load_from_checkpoint(
    checkpoint_path=os.path.join(
        root, "best_models_15122023", "seed_54321_20231214-203338.ckpt"
    )
)
plots_folder_name = "tests_new_structure"

In [ ]:
# Set-up seed
seed = 54321
seed_everything(seed, workers=True)
generator = torch.Generator().manual_seed(seed)

Apply model to unseen data #
tags_high = {"highway": ["motorway", "trunk", "primary", "secondary"]}
tags_build = {"building": ["yes"]}
tags_rail = {"railway": ["rail"]}

In [ ]:
min_x = min(x)
min_y = min(y)
max_x = max(x)
max_y = max(y)

In [ ]:
# Convert min_x and min_y to integers for use in formatting the latitude and longitude
west = math.floor(min_x)
south = math.floor(min_y)
east = math.ceil(max_x)
north = math.ceil(max_y)

In [ ]:
# Convert min_x and min_y to integers for use in formatting the latitude and longitude
lon = int(min_x)
lat = int(min_y)

In [ ]:
# Format the latitude and determine the season based on whether the latitude is positive or negative
if south >= 0:
    lat_f = "N{:02d}".format(south + 1)
    season = "summer"
else:
    lat_f = "S{:02d}".format(abs(south + 1))
    season = "winter"

In [ ]:
# Format the longitude based on whether it is positive or negative
if west >= 0:
    lon_f = "E{:03d}".format(west)
else:
    lon_f = "W{:03d}".format(abs(west))

In [ ]:
# Print the currently processed tile's coordinates
print(
    "Currently processed tile lat:{},{} lon:{},{}".format(min_y, max_y, min_x, max_x),
    flush=True,
)

In [ ]:
# Define bounds for the prediction tile
bounds = (min_x, min_y, max_x, max_y)
bounds_name = (int(min_x), int(min_y), int(max_x), int(max_y))
bounds_ltcoh = (
    math.floor(min_x),
    math.floor(min_y),
    math.ceil(max_x),
    math.ceil(max_y),
)

In [ ]:
tile_time_start = time.time()

In [ ]:
# Define the path for the PS prediction raster
file_name = os.path.join(
    experiment_dir, "{}{}".format(lat_f, lon_f), "{}{}_PS_pred.tif".format(lat_f, lon_f)
)

In [ ]:
# Check if prediction exists already
if os.path.exists(file_name):
    print("PS is already predicted for tile {}{} \n".format(lat_f, lon_f), flush=True)
    sys.exit()  # Stop the execution

In [ ]:
# If the prediction does not exist, start the prediction process
# Create the directory for storing outputs if it doesn't exist
dir_coord = os.path.join(experiment_dir, "{}{}".format(lat_f, lon_f))
if not os.path.exists(dir_coord):
    os.makedirs(dir_coord)

Infrastructure map #

In [ ]:
# Define the directory for the infrastructure map raster
infrmap_raster_dir = os.path.join(
    experiment_dir,
    "{}{}".format(lat_f, lon_f),
    "infr_map",
    "{}{}_infrs_map.tif".format(lat_f, lon_f),
)

In [ ]:
# Check if the infrastructure map already exists
if os.path.exists(infrmap_raster_dir):
    print(
        "Infrastructure map {}{} is already generated.".format(lat_f, lon_f),
        flush=True,
    )
# If the infrastructure map does not exist, start the generation process
else:
    generate_infrastructure_map(
        lat_f,
        lon_f,
        min_x,
        min_y,
        max_x,
        max_y,
        experiment_dir,
        clipped_osm_pbf,
        infrmap_raster_dir,
    )

Long-term coherence #

In [ ]:
# Start of long-term coherence section
start_time = time.time()

In [ ]:
# Define the path for the long-term coherence tile
coh_tile_path = "{coh_dir}/{lat}{lon}/coh/{lat}{lon}_{season}_vv_rho.tif".format(
    lat=lat_f, lon=lon_f, season=season, coh_dir=experiment_dir
)

In [ ]:
# Check if the long-term coherence tile already exists
# If not, try to download it
if os.path.exists(coh_tile_path):
    print(
        "Long-term coherence tile  {}{} is already downloaded.".format(lat_f, lon_f),
        flush=True,
    )
else:
    print(
        "\n Long-term coherence tile {}{} is missing! Trying to download \n".format(
            lat_f, lon_f
        ),
        flush=True,
    )

    try:
        os.system(
            f"aws s3 cp s3://sentinel-1-global-coherence-earthbigdata/data/tiles/{lat_f}{lon_f}/"
            f"{lat_f}{lon_f}_{season}_vv_rho.tif --no-sign-request "
            f"{experiment_dir}/{lat_f}{lon_f}/coh/{lat_f}{lon_f}_{season}_vv_rho.tif"
        )

        if os.path.exists(
            "{coh_dir}/{lat}{lon}/coh/{lat}{lon}_{season}_vv_rho.tif".format(
                lat=lat_f, lon=lon_f, season=season, coh_dir=experiment_dir
            )
        ):
            print("Download of {}{} tiles ended.".format(lat_f, lon_f))
        else:
            print("\n Download of tile {}{} failed. \n".format(lat_f, lon_f))

    except BaseException as error:
        print(
            "There is a problem with the download of the file. Please see following line to identify the error."
        )
        print("An exception occurred: {}".format(error))

    except Exception as e:
        print("An exception occurred: {}".format(e))

In [ ]:
# Calculate the total execution time for checking the long-term coherence file
end_time = time.time()
total_time = end_time - start_time
print(
    f"Total execution time for downloading the long term cohrence file: {total_time:.2f} seconds",
    flush=True,
)

Prepare input data for the model #

In [ ]:
# Create RasterDataset objects for coherence and infrastructure map images
imgs_coh = RasterDataset_imgs(
    paths=os.path.join(
        experiment_dir,
        "{}{}".format(lat_f, lon_f),
        "coh",
    ),
    transforms=scale_coh,
)
imgs_infrmap = RasterDataset_imgs(
    paths=os.path.join(experiment_dir, "{}{}".format(lat_f, lon_f), "infr_map")
)
# Combine the two RasterDataset objects
imgs_input = imgs_coh & imgs_infrmap

In [ ]:
# Set the dataset to predict
dataset_to_pred = imgs_input

In [ ]:
# Create a GridGeoSampler object for the dataset
sampler_pred = GridGeoSampler(dataset_to_pred, patch_size, patch_size - overlap)

In [ ]:
# Create dataloader
dataloader_interf = DataLoader(
    dataset=dataset_to_pred,
    batch_size=batch_size,
    sampler=sampler_pred,
    num_workers=0,
    collate_fn=stack_samples,
    generator=generator,
)

Run predictions #

In [ ]:
# Generate prediction chips
# disable randomness, dropout, etc...
model.eval()

In [ ]:
# Generate geotiff
# Get the pixel size and CRS of the dataset
pixel_size = dataset_to_pred.res
crs = dataset_to_pred.crs.to_epsg()

In [ ]:
# Generate prediction chips
start = time.time()  # Start measuring the time
chips_generator = georreferenced_chip_generator(
    dataloader_interf, model, crs, pixel_size
)
print("The time taken to predict was: ", time.time() - start, flush=True)

In [ ]:
# Save the prediction chips as a geotiff
start = time.time()  # Start measuring the time
merge_georeferenced_chips(chips_generator, file_name, overlap, bounds)
print(
    "The time taken to generate a georrefenced image and save it was: ",
    time.time() - start,
    flush=True,
)

In [ ]:
# Calculate the total execution time for processing one tile
tile_time_end = time.time()
tile_time = tile_time_end - tile_time_start
print(f"Time for processing one tile: {tile_time:.2f} seconds", flush=True)